<h1 style="text-align:center;line-height:1.5em;font-size:30px;">Data and Scripts <br>for Hydrological streamline Detection<br> Using U-NET Attention Module</h1>

<p style="text-align:center;font-size:12px;">
<strong>Department of Geography and Environmental Resources, Southern Illinois University Carbondale, IL, USA</strong><br>
<strong>Reference: 𝑍𝑒𝑤𝑒𝑖 𝑋𝑢 et. al. 2021</strong><br>
</p>
<hr>



In [ ]:
# Load all the dependencies
%reload_ext autoreload
%autoreload 2
import os
import sys
import random
import warnings
import numpy as np
from numpy import genfromtxt
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
np.random.seed(1337) # for reproducibility
# from tensorflow import set_random_seed
# set_random_seed(1337)
import numpy as np
np.random.seed(1337)

from itertools import chain
from keras.layers import Layer,UpSampling2D,concatenate
from tensorflow.keras.layers import BatchNormalization
from keras.layers.core import SpatialDropout2D, Activation
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers import Reshape, Permute, Input, add, multiply
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from tensorflow.keras.layers import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import tensorflow as tf
from keras.layers import dot
# To specify the GPU ID uncomment this block and  
# with K.tf.device('/gpu:0'): # specify the ID of GPU here (0: the first GPU)
#    config = tf.ConfigProto(intra_op_parallelism_threads=4,\
#           inter_op_parallelism_threads=4, allow_soft_placement=True,\
#           device_count = {'CPU' : 1, 'GPU' : 1})
#    session = tf.Session(config=config)
#    K.set_session(session)

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')



In [ ]:
# Use dice coefficient function as the loss function 
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(K.cast(y_true, 'float32'))
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2.0 * intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.0)

# Jacard coefficient
def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

# calculate loss value
def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)

# calculate loss value
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)



In [ ]:
def Residual_CNN_block(x, size, dropout=0.0, batch_norm=True):
    if K.image_data_format() == 'th':
        axis = 1
    else:
        axis = 3
    conv = Conv2D(size, (3, 3), padding='same')(x)
    if batch_norm is True:
        conv = BatchNormalization(axis=axis)(conv)
    conv = Activation('relu')(conv)
    conv = Conv2D(size, (3, 3), padding='same')(conv)
    if batch_norm is True:
        conv = BatchNormalization(axis=axis)(conv)
    conv = Activation('relu')(conv)
    conv = Conv2D(size, (3, 3), padding='same')(conv)
    if batch_norm is True:
        conv = BatchNormalization(axis=axis)(conv)
    conv = Activation('relu')(conv)
    return conv



In [ ]:
class multiplication(Layer):
    def __init__(self,inter_channel = None,**kwargs):
        super(multiplication, self).__init__(**kwargs)
        self.inter_channel = inter_channel
    def build(self,input_shape=None):
        self.k = self.add_weight(name='k',shape=(1,),initializer='zeros',dtype='float32',trainable=True)
    def get_config(self):
        base_config = super(multiplication, self).get_config()
        config = {'inter_channel':self.inter_channel}
        return dict(list(base_config.items()) + list(config.items()))  
    def call(self,inputs):
        g,x,x_query,phi_g,x_value = inputs[0],inputs[1],inputs[2],inputs[3],inputs[4]
        h,w,c = int(x.shape[1]),int(x.shape[2]),int(x.shape[3])
        x_query = K.reshape(x_query, shape=(-1,h*w, self.inter_channel//4))
        phi_g = K.reshape(phi_g,shape=(-1,h*w,self.inter_channel//4))
        x_value = K.reshape(x_value,shape=(-1,h*w,c))
        scale = dot([K.permute_dimensions(phi_g,(0,2,1)), x_query], axes=(1, 2))
        soft_scale = Activation('softmax')(scale)
        scaled_value = dot([K.permute_dimensions(soft_scale,(0,2,1)),K.permute_dimensions(x_value,(0,2,1))],axes=(1, 2))
        scaled_value = K.reshape(scaled_value, shape=(-1,h,w,c))        
        customize_multi = self.k * scaled_value
        layero = add([customize_multi,x])
        my_concat = Lambda(lambda x: K.concatenate([x[0], x[1]], axis=3))
        concate = my_concat([layero,g])
        return concate 
    def compute_output_shape(self,input_shape):
        ll = list(input_shape)[1]
        return (None,ll[1],ll[1],ll[3]*3)
    def get_custom_objects():
        return {'multiplication': multiplication}



In [ ]:
def attention_up_and_concatenate(inputs):
    g,x = inputs[0],inputs[1]
    inter_channel = g.get_shape().as_list()[3]
    g = Conv2DTranspose(inter_channel, (2,2), strides=[2, 2],padding='same')(g)
    x_query = Conv2D(inter_channel//4, [1, 1], strides=[1, 1], data_format='channels_last')(x)
    phi_g = Conv2D(inter_channel//4, [1, 1], strides=[1, 1], data_format='channels_last')(g)
    x_value = Conv2D(inter_channel//2, [1, 1], strides=[1, 1], data_format='channels_last')(x)
    inputs = [g,x,x_query,phi_g,x_value]
    concate = multiplication(inter_channel)(inputs)
    return concate



In [ ]:
class multiplication2(Layer):
    def __init__(self,inter_channel = None,**kwargs):
        super(multiplication2, self).__init__(**kwargs)
        self.inter_channel = inter_channel
    def build(self,input_shape=None):
        self.k = self.add_weight(name='k',shape=(1,),initializer='zeros',dtype='float32',trainable=True)
    def get_config(self):
        base_config = super(multiplication2, self).get_config()
        config = {'inter_channel':self.inter_channel}
        return dict(list(base_config.items()) + list(config.items()))  
    def call(self,inputs):
        g,x,rate = inputs[0],inputs[1],inputs[2]
        scaled_value = multiply([x, rate])
        att_x =  self.k * scaled_value
        att_x = add([att_x,x])
        my_concat = Lambda(lambda x: K.concatenate([x[0], x[1]], axis=3))
        concate = my_concat([att_x, g])
        return concate 
    def compute_output_shape(self,input_shape):
        ll = list(input_shape)[1]
        return (None,ll[1],ll[1],ll[3]*2)
    def get_custom_objects():
        return {'multiplication2': multiplication2}



In [ ]:
def attention_up_and_concatenate2(inputs):
    g, x = inputs[0],inputs[1]
    inter_channel = g.get_shape().as_list()[3]
    g = Conv2DTranspose(inter_channel//2, (3,3), strides=[2, 2],padding='same')(g)
    g = Conv2D(inter_channel//2, [1, 1], strides=[1, 1], data_format='channels_last')(g)
    theta_x = Conv2D(inter_channel//4, [1, 1], strides=[1, 1], data_format='channels_last')(x)
    phi_g = Conv2D(inter_channel//4, [1, 1], strides=[1, 1], data_format='channels_last')(g)
    f = Activation('relu')(add([theta_x, phi_g]))
    psi_f = Conv2D(1, [1, 1], strides=[1, 1], data_format='channels_last')(f)
    rate = Activation('sigmoid')(psi_f)
    concate =  multiplication2()([g,x,rate])
    return concate



In [ ]:
def UNET_224(weights=None):
    inputs = Input((IMG_WIDTH, IMG_WIDTH, INPUT_CHANNELS))
    filters = 32
    last_dropout = 0.2
# convolutiona and pooling level 1
    conv_224 = Residual_CNN_block(inputs,filters)
    pool_112 = MaxPooling2D(pool_size=(2, 2))(conv_224)
# convolutiona and pooling level 2
    conv_112 = Residual_CNN_block(pool_112,2*filters)
    pool_56 = MaxPooling2D(pool_size=(2, 2))(conv_112)
# convolutiona and pooling level 3
    conv_56 = Residual_CNN_block(pool_56,4*filters)
    pool_28 = MaxPooling2D(pool_size=(2, 2))(conv_56)
# convolutiona and pooling level 4
    conv_28 = Residual_CNN_block(pool_28,8*filters)
    pool_14 = MaxPooling2D(pool_size=(2, 2))(conv_28)
# convolutiona and pooling level 5
    conv_14 = Residual_CNN_block(pool_14,16*filters)
    pool_7 = MaxPooling2D(pool_size=(2, 2))(conv_14)
# Conlovlution and feature concatenation
    conv_7 = Residual_CNN_block(pool_7,32*filters)
# Upsampling with convolution 
    up_14 = attention_up_and_concatenate([conv_7, conv_14]) 
    up_conv_14 = Residual_CNN_block(up_14,16*filters)
# Upsampling with convolution 2
    up_28 = attention_up_and_concatenate([up_conv_14, conv_28])
    up_conv_28 = Residual_CNN_block(up_28,8*filters)
# Upsampling with convolution 3
    up_56 = attention_up_and_concatenate2([up_conv_28, conv_56])
    up_conv_56 = Residual_CNN_block(up_56,4*filters)
# Upsampling with convolution 4
    up_112 = attention_up_and_concatenate2([up_conv_56, conv_112])
    up_conv_112 = Residual_CNN_block(up_112,2*filters)
# Upsampling with convolution 5
    up_224 = attention_up_and_concatenate2([up_conv_112, conv_224])
    #up_224 = attention_up_and_concatenate2(up_conv_112, conv_224)
    up_conv_224 = Residual_CNN_block(up_224,filters,dropout = last_dropout)
# 1 dimensional convolution and generate probabilities from Sigmoid function
    conv_final = Conv2D(OUTPUT_MASK_CHANNELS, (1, 1))(up_conv_224)  
    conv_final = Activation('sigmoid')(conv_final)
# Generate model
    model = Model(inputs, conv_final, name="UNET_224")
    return model



In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# scenario = raw_input("Please specify the scenario (up,down,left,or right):")
# aug = v2
# read in training and validation data
train_data = np.load(r"C:\Users\mikeb\Desktop\New data\train2\data\train_data_augv2.npy")#[:2000]
train_label = np.load(r"C:\Users\mikeb\Desktop\New data\train2\data\train_label_augv2.npy")#[:,:,:,np.newaxis]#[:,:,:,np.newaxis]#[:2000]
# X_Validation = np.load(r"C:\Users\mikeb\Desktop\orig\data\train_data_augv2.npy")[:700]
# Y_Validation = np.load(r"C:\Users\mikeb\Desktop\orig\data\train_label_augv2.npy")#[:,:,:,np.newaxis][:700]
# np.save(r"C:\Users\mikeb\Desktop\orig\train_label_augP2_.npy",Y_Validation[:,:,:,np.newaxis])



# Assuming your original training data and labels are stored in variables `train_data` and `train_label`

# Set a random seed for reproducibility
np.random.seed(42)

# Reshape the training labels to match the desired split shape
# train_label = train_label.reshape(245, -1)

# Split the training data and labels into training and validation sets
train_data, vali_data, train_label, vali_label = train_test_split(train_data, train_label, test_size=20, random_state=42)

# Verify the shapes of the training and validation sets
print(train_data.shape)  # (225, 224, 224, 5)
print(vali_data.shape)  # (20, 224, 224, 5)
print(train_label.shape)  # (225, 224, 224, 1)
print(vali_label.shape)  # (20, 224, 224, 1)





s = np.arange(train_data.shape[0])
np.random.shuffle(s)
train_data = train_data[s]
train_label = train_label[s]

# print(X_train.shape) 
# print(Y_train.shape)
# print(X_Validation.shape)
# print(Y_Validation.shape)

patch_size = 224

IMG_WIDTH = patch_size
IMG_HEIGHT = patch_size
# Number of feature channels 
INPUT_CHANNELS = 5
# Number of output masks (1 in case you predict only one type of objects)
OUTPUT_MASK_CHANNELS = 1
maxepoch = 300
# hyperparameters
learning_rate =0.0000359
patience = 20


In [ ]:
import numpy as np
# scenario = raw_input("Please specify the scenario (up,down,left,or right):")
# aug = '_aug_'+scenario 
# read in training and validation data
X_train = np.load(r"C:\Users\mikeb\Desktop\di_gen\data\train_data_augv2.npy")#[:2000]
Y_train = np.load(r"C:\Users\mikeb\Desktop\di_gen\data\train_label_augv2.npy")#[:,:,:,np.newaxis]#[:,:,:,np.newaxis]#[:2000]
X_Validation = np.load(r"C:\Users\mikeb\Desktop\di_gen\valid\data\train_data_augv2.npy")#[:700]
Y_Validation = np.load(r"C:\Users\mikeb\Desktop\di_gen\valid\data\train_label_augv2.npy")
s = np.arange(X_train.shape[0])
np.random.shuffle(s)
X_train = X_train[s]
Y_train = Y_train[s]

print(X_train.shape)
print(Y_train.shape)
print(X_Validation.shape)
print(Y_Validation.shape)

patch_size = 224

IMG_WIDTH = patch_size
IMG_HEIGHT = patch_size
# Number of feature channels 
INPUT_CHANNELS = 5
# Number of output masks (1 in case you predict only one type of objects)
OUTPUT_MASK_CHANNELS = 1
maxepoch = 300
# hyperparameters
learning_rate =0.0000359
patience = 20



In [ ]:
# create the CNN
model = UNET_224()
model.compile(optimizer=Adam(lr=learning_rate),loss = dice_coef_loss,metrics=[dice_coef,'accuracy'])
callbacks = [
        ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=patience, min_lr=1e-9, verbose=1, mode='min'),
        EarlyStopping(monitor='val_loss', patience=patience, verbose=0),
        ModelCheckpoint(r'C:\Users\mikeb\Desktop\last\result\attention2vorg.h5', monitor='val_loss', save_best_only=True, verbose=0),
    ]

results_03 = model.fit(train_data, train_label, validation_data=(vali_data,vali_label), batch_size=16, epochs=maxepoch, callbacks=callbacks)
from keras.models import load_model
import pickle
# save the intermdediate results and training statistics
with open(r'C:\Users\mikeb\Desktop\last\result\history_attention_U_netorg.pickle', 'wb') as file_pi:
    pickle.dump(results_03.history, file_pi, protocol=2)
# save the model5
model.save()

In [ ]:
# create the CNN
model = UNET_224()
model.compile(optimizer=Adam(lr=learning_rate),loss = dice_coef_loss,metrics=[dice_coef,'accuracy'])
callbacks = [
        ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=patience, min_lr=1e-9, verbose=1, mode='min'),
        EarlyStopping(monitor='val_loss', patience=patience, verbose=0),
        ModelCheckpoint(r"C:\Users\mikeb\Desktop\di_gen\result\attention2.h5", monitor='val_loss', save_best_only=True, verbose=0),
    ]

results_03 = model.fit(X_train, Y_train, validation_data=(X_Validation,Y_Validation), batch_size=16, epochs=maxepoch, callbacks=callbacks)
from keras.models import load_model
import pickle
# save the intermdediate results and training statistics
with open(r"C:\Users\mikeb\Desktop\di_gen\result\historyattention_U_net.pickle", 'wb') as file_pi:
    pickle.dump(results_03.history, file_pi, protocol=2)
# save the model
model.save(r"C:\Users\mikeb\Desktop\di_gen\result\attUnet.h5")


In [ ]:
# load weights into new model
from keras.models import load_model

newmodel = load_model(r"C:\Users\mikeb\Downloads\attention_U_net_c.h5", custom_objects={'multiplication': multiplication,'multiplication2': multiplication2,"dice_coef_loss":dice_coef_loss, "dice_coef":dice_coef})
# Reload the model and save the predicted labels.
X_test = np.load(r"C:\Users\mikeb\Desktop\valid_data\data\pred.npy")
preds_test = newmodel.predict(X_test)
preds_test_t = (preds_test > 0.5).astype(np.uint8)
# np.save(r"C:\Users\mikeb\Desktop\ediited_dem\pred_result.npy",preds_test_t)


In [ ]:
import copy
import random
import sys
import numpy as np
import tifffile as tiff
buf =30

preds_test_mod = preds_test_t#np.load(r"C:\Users\mikeb\Desktop\ediited_dem\pred_result.npy")
dim = np.load(r"C:\Users\mikeb\Desktop\valid_data\mask.npy").shape
numr = dim[0]//(224 - buf*2)
numc = dim[1]//(224 - buf*2)
count = -1
for i in range(numr):
    for j in range(numc):
        count += 1    
        temp = preds_test_mod[count][buf:-buf,buf:-buf]
        if j == 0:
            rows = temp
        else:
            rows = np.concatenate((rows,temp),axis = 1)
    if i == 0:
        prediction_map = copy.copy(rows)
    else:
        prediction_map = np.concatenate((prediction_map,rows),axis = 0)
prediction_map = prediction_map[:,:,0]
# fig= plt.figure(figsize=(20,15))

In [ ]:
# # Generate prediction map
# from osgeo import gdal 
import numpy as np
import cv2
import numpy as np
import copy
import matplotlib.pyplot as plt
# mask = np.load(r"C:\Users\mikeb\Desktop\last\masklast.npy")
fig= plt.figure(figsize=(20,15))
# prediction_map= cv2.resize(prediction_map, (6075,7850))
# Plot the map
prediction_map = prediction_map #*mask
plt.subplot(1,2, 1)
plt.title('result',fontsize = 16)
plt.imshow(prediction_map*255,cmap='gray',vmin=0, vmax=1)

reference = np.load(r"C:\Users\mikeb\Desktop\valid_data\data\label.npy").astype(np.uint8)
plt.subplot(1,2,2)
plt.title('Reference map',fontsize = 16)
plt.imshow(reference, cmap = 'gray')
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import os
import pickle
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.metrics import confusion_matrix

"""
plot_history(): reads Keras result and ogenerate figure of Loss, Dice Coefficient, and Accuracy.
"""
def plot_history(history):
    loss_list = [s for s in history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.keys() if 'loss' in s and 'val' in s]
    dice_list = [s for s in history.keys() if 'dice' in s and 'val' not in s]
    val_dice_list = [s for s in history.keys() if 'dice' in s and 'val' in s]
    acc_list = [s for s in history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history[loss_list[0]]) + 1)
    
    ## Loss
    plt.subplot(1,3,1)
    for l in loss_list:
        plt.plot(epochs, history[l], 'b', label='Training loss (' + str(str(format(history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history[l], 'g', label='Validation loss (' + str(str(format(history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Dice Coefficient
    plt.subplot(1,3,2)
    for l in dice_list:
        plt.plot(epochs, history[l], 'b', label='Training Dice Coefficient (' + str(format(history[l][-1],'.5f'))+')')
    for l in val_dice_list:    
        plt.plot(epochs, history[l], 'g', label='Validation Dice Coefficient (' + str(format(history[l][-1],'.5f'))+')')

    plt.title('Dice Coefficient')
    plt.xlabel('Epochs')
    plt.ylabel('Dice Coefficient')
    plt.legend()
    
    ## Accuracy
    plt.subplot(1,3,3)
    for l in acc_list:
        plt.plot(epochs, history[l], 'b', label='Training accuracy (' + str(format(history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history[l], 'g', label='Validation accuracy (' + str(format(history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
infile = open(r"C:\Users\mikeb\Downloads\attention_left.pickle",'rb')
new_dict = pickle.load(infile)
infile.close()
plt.figure(figsize=(20,5))
plot_history(new_dict)

In [ ]:
import copy
import random
import sys
import numpy as np

buf =30
"""
plot_confusion_matrix(): prints and plots the confusion matrix. Normalization can be applied by setting `normalize=True`.
"""
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# preds_test_mod = np.load(r"C:\Users\mikeb\Desktop\last\test_result.npy")

# dim = np.load(r"C:\Users\mikeb\Desktop\last\ref.npy").shape
# numr = dim[0]//(224 - buf*2)
# numc = dim[1]//(224 - buf*2)
# count = -1
# for i in range(numr):
#     for j in range(numc):
#         count += 1    
#         temp = preds_test_mod[count][buf:-buf,buf:-buf]
#         if j == 0:
#             rows = temp
#         else:
#             rows = np.concatenate((rows,temp),axis = 1)
#     if i == 0:
#         prediction_map = copy.copy(rows)
#     else:
#         prediction_map = np.concatenate((prediction_map,rows),axis = 0)
# prediction_map = prediction_map[:,:,0]
# # fig= plt.figure(figsize=(20,15))
# # plt.imshow(prediction_map*255,cmap='gray',vmin=0, vmax=255)
# # plt.show()

# #print(np.unique(prediction_map))

# mask
mask = np.load(r"C:\Users\mikeb\Desktop\valid_data\mask.npy")[:prediction_map.shape[0],:prediction_map.shape[1]]
[lr,lc] = np.where(mask == 1)

# Read reference data
groundtruthlist = np.load(r"C:\Users\mikeb\Desktop\valid_data\data\label.npy")[:prediction_map.shape[0],:prediction_map.shape[1]][lr,lc]
predictionlist = prediction_map[lr,lc]

# print(np.unique(groundtruthlist*255))
# print(np.unique(predictionlist))
# print(type(groundtruthlist))
# print(groundtruthlist)
# print(type(predictionlist))
# print(predictionlist)

cm = confusion_matrix((groundtruthlist).astype(int), predictionlist)
plot_confusion_matrix(cm,classes=["Non-streams","Streams"])


In [ ]:
# Statistics
from sklearn.metrics import f1_score, precision_score,recall_score
#print(f1_score(groundtruthlist, predictionlist, average='macro'))
print('F1 score of stream: '+str(f1_score(groundtruthlist, predictionlist,pos_label=1)))
print('F1 score of nonstream: '+str(f1_score(groundtruthlist, predictionlist,pos_label=0)))
print('Precision of stream: '+str(precision_score(groundtruthlist, predictionlist,pos_label=1)))
print('Precision of nonstream: '+str(precision_score(groundtruthlist, predictionlist,pos_label=0)))
print('Recall of stream: '+str(recall_score(groundtruthlist, predictionlist,pos_label=1)))
print('Recall of nonstream: '+str(recall_score(groundtruthlist, predictionlist,pos_label=0)))